# Imports and Functions

In [1]:
from sunlab.environment.base.cpu import *
from sunlab.environment.base.extras import *
from sunlab.globals import FILES
from sunlab.sunflow import *

In [2]:
import seaborn as sns

# Setup

In [3]:
model = load_aae(FILES['PRETRAINED_MODEL_DIR'], MaxAbsScaler)

In [4]:
training_data = process_and_load_dataset(FILES['TRAINING_DATASET'], FILES['PRETRAINED_MODEL_DIR'])

# Runtime Parameters

In [5]:
SAVE_EXTRAS = False

In [6]:
g_Base = lambda *args: DIR_ROOT + "Figures/Theory/SHAP/"

# Plots

In [7]:
import shap
shap.initjs()

In [8]:
def f(X):
    return model.encoder(model.scaler(X)).numpy()[:,0].flatten()
def g(X):
    return model.encoder(model.scaler(X)).numpy()[:,1].flatten()

In [9]:
X = training_data.dataframe[training_data.data_columns]

In [10]:
skm = shap.kmeans(X.iloc[:,:], 1000)
skm_df = pd.DataFrame(skm.data, columns=skm.group_names)
skm_df.head(10)

,Area,MjrAxisLength,MnrAxisLength,Eccentricity,ConvexArea,EquivDiameter,Solidity,Extent,Perimeter,ConvexPerim,FibLen,InscribeR,BlebLen
0,211.0,36.722211,8.711188,0.968852,266.0,16.390654,0.794020,0.411111,80.056,77.534,38.627419,3.605551,23.384773
1,1540.0,65.363788,37.708153,0.766119,2216.0,44.280796,0.694805,0.425000,231.457,184.959,88.911728,13.453624,47.313713
2,2938.0,76.557791,62.183505,0.583320,4453.0,61.161898,0.659780,0.442470,339.400,259.236,120.740158,21.189621,39.556355
3,721.0,48.830231,23.929616,0.835715,982.0,30.298609,0.734375,0.482759,149.410,123.542,60.112717,8.246211,34.242641
4,355.0,50.920947,13.648738,0.956439,574.0,21.260293,0.618976,0.338875,132.248,111.027,58.041645,4.472136,37.142139
5,1247.0,50.634416,35.319727,0.687690,1532.0,39.846326,0.814039,0.564185,174.361,145.184,66.426430,12.806249,30.656855
6,2217.0,65.224865,50.297465,0.597780,2867.0,53.129766,0.774809,0.515038,234.849,200.950,91.840652,17.888544,33.041626
7,4532.0,98.022152,69.657870,0.702982,6137.0,75.407478,0.733624,0.519763,397.931,290.572,144.237671,22.627417,79.669083
8,112.0,17.898167,9.589005,0.813087,137.0,11.941643,0.817185,0.543478,46.156,43.195,19.970560,3.162278,9.242640
9,588.0,36.906965,22.680018,0.748646,704.0,27.361741,0.835671,0.566154,111.225,99.509,45.041641,8.602325,19.727921


In [11]:
X = skm_df

In [12]:
N_Tr = 100
N_Val = 100
Tr = X.iloc[:N_Tr,:]
Va = X.iloc[N_Tr:N_Tr+N_Val,:]
explainer_f = shap.KernelExplainer(f, Tr)
explainer_g = shap.KernelExplainer(g, Tr)
shap_values_f = explainer_f.shap_values(Va, nsamples=250)
shap_values_g = explainer_g.shap_values(Va, nsamples=250)
display(shap.force_plot(explainer_f.expected_value, shap_values_f, Va))
display(shap.force_plot(explainer_g.expected_value, shap_values_g, Va))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
def f_Area(X):
    return model.scaler.scaler.inverse_transform(model.decoder(X).numpy())[:,0].flatten()

In [14]:
X = pd.DataFrame(model.encoder(training_data.dataset).numpy(), columns=["Latent-0","Latent-1"])

In [15]:
Tr = X.iloc[50*0:50*(0+1),:]
Va = X.iloc[50*1:50*(1+1),:]
explainer_f = shap.KernelExplainer(f_Area, Tr)
shap_values_f = explainer_f.shap_values(Va, nsamples=50)
display(shap.force_plot(explainer_f.expected_value, shap_values_f, Va))

  0%|          | 0/50 [00:00<?, ?it/s]